Day 3 - Today, we work on Milstein Scheme for the GBM (Geometric Brownian Motion), its almost similar but we'll compare the 2 vectorized functions to see differences, since this one is better in term of accuracy